In [1]:
import kfp.dsl as kfp
from kfp import dsl
from kfp import Client
from kfp import components
import yaml


In [2]:
def model_serving(cpu_request, memory_request):
    """
    Create kserve instance
    """
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1TFServingSpec
    from datetime import datetime
    from kubernetes.client import V1Container, V1EnvVar, V1ResourceRequirements

    namespace = utils.get_default_target_namespace()

    now = datetime.now()
    v = now.strftime("%Y-%m-%d--%H-%M-%S")

    name='ancestry-{}'.format(v)
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       containers=[V1Container(image='hungcao0402/ancestry:latest',
                                                                name='ancestry',
                                                                resources=V1ResourceRequirements(requests={'cpu': cpu_request, 'memory': memory_request})                                                               
                                                   
                                        )]
                                   )
    ))
    KServe = KServeClient()
    KServe.create(isvc)     
    
comp_model_serving = components.func_to_container_op(model_serving, packages_to_install=["kserve==0.10", 'kubernetes'])

In [3]:
@kfp.pipeline(
  name='pipeline-ancestry',
  description='Kubeflow pipelines SDK'
)
def ancestry_pipeline(cpu_request='1m', memory_request='1Gi'):    
    comp_model_serving(cpu_request, memory_request).execution_options.caching_strategy.max_cache_staleness = "P0D"

if __name__ == "__main__":
    import kfp.compiler as compiler
    compiler.Compiler().compile(ancestry_pipeline, 'ancestry_pipeline.yaml')
